# imports/setup

# backtester

# trader

# backtest run

In [1]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

### setup

In [2]:
def calculate_starfruit_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_amethysts_fair(order_depth):
    return 10000

### RUN FROM LOG

In [3]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math
import io
from datamodel import Listing, ConversionObservation
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation
from round4_v3_bt import Trader
from backtester import Backtester
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math


In [4]:
listings = {
    'AMETHYSTS': Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS'),
    'ORCHIDS': Listing(symbol='ORCHIDS', product='ORCHIDS', denomination='SEASHELLS'),
    'CHOCOLATE': Listing(symbol='CHOCOLATE', product='CHOCOLATE', denomination='SEASHELLS'),
    'STRAWBERRIES': Listing(symbol='STRAWBERRIES', product='STRAWBERRIES', denomination='SEASHELLS'),
    'ROSES': Listing(symbol='ROSES', product='ROSES', denomination='SEASHELLS'),
    'GIFT_BASKET': Listing(symbol='GIFT_BASKET', product='GIFT_BASKET', denomination='SEASHELLS'),
    'COCONUT': Listing(symbol='COCONUT', product="COCONUT", denomination="SEASHELLS"),
    "COCONUT_COUPON": Listing(symbol='COCONUT_COUPON', product='COCONUT_COUPON', denomination="SEASHELLS")
}

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20,
    'CHOCOLATE': 250,
    'STRAWBERRIES': 350,
    'ROSES': 60,
    'GIFT_BASKET': 60,
    'COCONUT': 300,
    'COCONUT_COUPON': 600
}

fair_calculations = {
}

In [5]:
market_data, trade_history = _process_data_("./clean_logs/results_round3.log")
trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, f"backtests/results_round3.log")
backtester.run()
print(backtester.pnl)

{'AMETHYSTS': 7800.0, 'STARFRUIT': 7836.0, 'ORCHIDS': 0.0, 'CHOCOLATE': 4664.0, 'STRAWBERRIES': 4578.0, 'ROSES': 39252.0, 'GIFT_BASKET': 25419.0, 'COCONUT': 0, 'COCONUT_COUPON': 0}


### RUN FROM DATABOTTLE

In [6]:
for i in range(1, 4):
    # with fair prediction
    day = i
    market_data = pd.read_csv(f"./round-4-island-data-bottle/prices_round_4_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./round-4-island-data-bottle/trades_round_4_day_{day}_nn.csv", sep=";", header=0)

    trader = Trader()
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, f"clean_data_logs/trade_history_day_{i}.log")
    backtester.run()
    print(backtester.pnl)

KeyError: 'ORCHIDS'

`{'AMETHYSTS': 14554, 'STARFRUIT': 14144.5}`

In [ ]:
day = 1
market_data1 = pd.read_csv(f"./round-4-island-data-bottle/prices_round_4_day_{day}.csv", sep=";", header=0)
trade_history1 = pd.read_csv(f"./round-4-island-data-bottle/trades_round_4_day_{day}_nn.csv", sep=";", header=0)

day = 2
market_data2 = pd.read_csv(f"./round-4-island-data-bottle/prices_round_4_day_{day}.csv", sep=";", header=0)
trade_history2 = pd.read_csv(f"./round-4-island-data-bottle/trades_round_4_day_{day}_nn.csv", sep=";", header=0)
market_data2['timestamp'] = market_data2['timestamp'] + 1000000
trade_history2['timestamp'] = trade_history2['timestamp'] + 1000000

day = 3
market_data3 = pd.read_csv(f"./round-4-island-data-bottle/prices_round_4_day_{day}.csv", sep=";", header=0)
trade_history3 = pd.read_csv(f"./round-4-island-data-bottle/trades_round_4_day_{day}_nn.csv", sep=";", header=0)
market_data3['timestamp'] = market_data3['timestamp'] + 2000000
trade_history3['timestamp'] = trade_history3['timestamp'] + 2000000

market_data = pd.concat([market_data1, market_data2, market_data3])
trade_history = pd.concat([trade_history1, trade_history2, trade_history3])



# backtest gridsearch

In [ ]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [ ]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

### run

In [ ]:
backtest_dir = "backtestruns"

param_grid = {
    "COCONUT_COUPON": {
        "mean_volatility": [0.15959997370608378],
        "threshold": [0.00163],
        "strike": [10000],
        "starting_time_to_expiry": [247 / 250],
        "std_window": [30],
        "zscore_threshold": np.arange(4.8,5.3,0.1)
    },
}

# param_grid = {
#     "COCONUT_COUPON": {
#         "mean_volatility": [0.15959997370608378],
#         "threshold": [0.00163],
#         "strike": [10000],
#         "starting_time_to_expiry": [247 / 250],
#         "std_window": [6],
#         "zscore_threshold": [20]
#     },
# }

trader = Trader()

coconut_coupon_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "COCONUT_COUPON")
print("COCONUT_COUPON results:")
for params, pnl in coconut_coupon_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)


Running backtests for COCONUT_COUPON:   0%|          | 0/5 [00:00<?, ?backtest/s]Running backtests for COCONUT_COUPON: 100%|██████████| 5/5 [03:25<00:00, 41.05s/backtest]

COCONUT_COUPON results:
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 30, 'zscore_threshold': 4.8}
pnl: 434053.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 30, 'zscore_threshold': 4.8999999999999995}
pnl: 432395.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 30, 'zscore_threshold': 4.999999999999999}
pnl: 439458.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 30, 'zscore_threshold': 5.099999999999999}
pnl: 439686.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 30, 'zscore_threshold': 5.199999999999998}
pnl: 432930.0


## analyze

In [ ]:

coconut_coupon_results.sort(key=lambda x: x[1], reverse=True)
print("Top 5 PnLs in COCONUT_COUPON results:")
for params, pnl in coconut_coupon_results[:5]: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)



Top 5 PnLs in COCONUT_COUPON results:
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 30, 'zscore_threshold': 7}
pnl: 431250.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 15, 'zscore_threshold': 7}
pnl: 428369.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 20, 'zscore_threshold': 9}
pnl: 428119.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 20, 'zscore_threshold': 7}
pnl: 427421.0
{'mean_volatility': 0.15959997370608378, 'threshold': 0.00163, 'strike': 10000, 'starting_time_to_expiry': 0.988, 'std_window': 25, 'zscore_threshold': 8}
pnl: 426677.0


In [ ]:

def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split('-', 1)
        params = dict(param.split('=') for param in params_str.split('-'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

In [ ]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math
import io
from datamodel import Listing, ConversionObservation
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

In [ ]:
class Product:
    AMETHYSTS = "AMETHYSTS"
    STARFRUIT = "STARFRUIT"
    ORCHIDS = "ORCHIDS"
    GIFT_BASKET = "GIFT_BASKET"
    CHOCOLATE = "CHOCOLATE"
    STRAWBERRIES = "STRAWBERRIES"
    ROSES = "ROSES"
    SYNTHETIC = "SYNTHETIC"
    SPREAD = "SPREAD"

def create_params(sma_window, std_window, zscore_threshold, target_position):
    return {
        Product.AMETHYSTS: {
            "fair_value": 10000,
            "take_width": 1,
            "clear_width": 0.5,
            "volume_limit": 0,
        },
        Product.STARFRUIT: {
            "take_width": 1,
            "clear_width": 0,
            "prevent_adverse": True,
            "adverse_volume": 15,
            "reversion_beta": -0.229,
            "starfruit_min_edge": 2,
        },
        Product.ORCHIDS: {
            "make_edge": 2,
            "make_min_edge": 1,
            "make_probability": 0.566,
            "init_make_edge": 2,
            "min_edge": 0.5,
            "volume_avg_timestamp": 5,
            "volume_bar": 75,
            "dec_edge_discount": 0.8,
            "step_size": 0.5,
        },
        Product.SPREAD:{
            "default_spread_mean": 379.50439988484237,
            "default_spread_std": 76.07966,
            "spread_sma_window": sma_window,
            "spread_std_window": std_window,
            "zscore_threshold": zscore_threshold,
            "target_position": target_position
        }
    }

In [ ]:
from backtester import Backtester
from round3_v1_simple_strat import Trader
from tqdm import tqdm
import itertools

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

listings = {
    'AMETHYSTS': Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS'),
    'ORCHIDS': Listing(symbol='ORCHIDS', product='ORCHIDS', denomination='SEASHELLS'),
    'CHOCOLATE': Listing(symbol='CHOCOLATE', product='CHOCOLATE', denomination='SEASHELLS'),
    'STRAWBERRIES': Listing(symbol='STRAWBERRIES', product='STRAWBERRIES', denomination='SEASHELLS'),
    'ROSES': Listing(symbol='ROSES', product='ROSES', denomination='SEASHELLS'),
    'GIFT_BASKET': Listing(symbol='GIFT_BASKET', product='GIFT_BASKET', denomination='SEASHELLS'),
}

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20,
    'CHOCOLATE': 250,
    'STRAWBERRIES': 350,
    'ROSES': 60,
    'GIFT_BASKET': 60
}

fair_calculations = {}

market_data0, trade_history0 = _process_data_('./clean_log/trade_history_day_0.log')
market_data1, trade_history1 = _process_data_('./clean_log/trade_history_day_1.log')
market_data2, trade_history2 = _process_data_('./clean_log/trade_history_day_2.log')
market_data = market_data0
trade_history = trade_history0

sma_windows = [1500]
std_windows = [45]
zscore_thresholds = [9]
target_positions = [58]


for sma_window, std_window, zscore_threshold, target_position in tqdm(itertools.product(sma_windows, std_windows, zscore_thresholds, target_positions)):
    total_pnl = 0
    params = create_params(sma_window, std_window, zscore_threshold, target_position)
    trader = Trader(params=params)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "simple_strat_backtest_no_clear_test_exceed.log")
    backtester.run()
    pnl = sum(float(pnl) for pnl in backtester.pnl.values())
    total_pnl += pnl
    print(f"sma_window: {sma_window}, std_window: {std_window}, zscore_threshold: {zscore_threshold}, target_position: {target_position}, pnl: {total_pnl}")
    print("="*80)



1it [00:47, 47.13s/it]

sma_window: 1500, std_window: 45, zscore_threshold: 9, target_position: 58, pnl: 46695.0
